# Convolutional Networks Architectures

In [2]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
def evaluate_accuracy(data_iter, net, device):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum +=(net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [5]:
def train(net, train_iter, test_iter, optimizer, num_epochs, device):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
                print(f"Step {i}, time since epoch: {time.time() - start:.3f}. "
                      f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device), device)
        print(f"epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}" \
              f", test acc {test_acc:.3f}, time {time.time() - start:.1f} sec")    


## Dataset

In [6]:
BATCH_SIZE = 256
transforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('./datas', train=True, transform=transforms, download=True)
test_dataset = tv.datasets.MNIST('./datas', train=False, transform=transforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## AlexNet

In [7]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [8]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [9]:
lr, num_epochs = 0.001, 1
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, optimizer, num_epochs, device)

Step 0, time since epoch: 0.338. Train acc: 0.078. Train Loss: 2.302
Step 10, time since epoch: 2.172. Train acc: 0.106. Train Loss: 2.333
Step 20, time since epoch: 4.089. Train acc: 0.149. Train Loss: 2.281
Step 30, time since epoch: 6.158. Train acc: 0.258. Train Loss: 2.057
Step 40, time since epoch: 8.083. Train acc: 0.356. Train Loss: 1.799
Step 50, time since epoch: 9.890. Train acc: 0.437. Train Loss: 1.582
Step 60, time since epoch: 11.674. Train acc: 0.496. Train Loss: 1.419
Step 70, time since epoch: 13.484. Train acc: 0.544. Train Loss: 1.286
Step 80, time since epoch: 15.267. Train acc: 0.585. Train Loss: 1.175
Step 90, time since epoch: 17.077. Train acc: 0.620. Train Loss: 1.080
Step 100, time since epoch: 18.935. Train acc: 0.649. Train Loss: 1.001
Step 110, time since epoch: 20.736. Train acc: 0.672. Train Loss: 0.933
Step 120, time since epoch: 22.562. Train acc: 0.693. Train Loss: 0.877
Step 130, time since epoch: 24.374. Train acc: 0.711. Train Loss: 0.828
Step 140,

Хороший результат